In [1]:
"""Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.

Attributes
----------

"""

'Output Transactions\n\nThis notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.\n\nAttributes\n----------\n\n'

# Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.


## Setup LUSID




In [2]:
# Import general purpose packages
import json
import os
import pandas as pd

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.utilities import create_scope_id

# Set pandas options
pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

ConnectionError: HTTPSConnectionPool(host='lusid-fbn-ci.okta.com', port=443): Max retries exceeded with url: /oauth2/aus49h9b8rQPvouCH2p7/v1/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f5269dbf9d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

## Load source transactions


In [ ]:
# Create a scope and portfolio code
scope = create_scope_id()
portfolio_code = "EQUITY_UK"

In [ ]:
df = pd.read_csv("data/equity_transactions.csv")
df

## Load instruments

Create the instruments and add on ISIN and Sedol as identifiers

In [ ]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ticker",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

## Create portfolio

In [ ]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])
print(succ,failed)

## Load in transactions

In [ ]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",        
    },
    "optional": {},
    "properties": [],
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(failed)}])

## Check holdings

In [ ]:
# Define the transaction portfolio API
txn_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

In [ ]:
# Load a mapping file for DataFrame headers for the get holdings response
with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

## Create corporate actions source

The corporate actions source is a container for holding corporate transactions. In this section we:
- Create a new corporate actions source
- Assign corporate actions source to our portfolio

In [ ]:
ca_source_code = "ca_demo"

# Create first corporate action source

try:
    source_request = models.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_source_code,
        display_name=ca_source_code,
        description="Corporate Actions source for Output Transactions notebook",
    )
    
    source_result = api_factory.build(
        lusid.api.CorporateActionSourcesApi
    ).create_corporate_action_source(create_corporate_action_source_request=source_request)
    
except:
    pass

In [ ]:
# Assign corporate actions source to our portfolio

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_portfolio_details(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-01T00:00:00+00:00",
    create_portfolio_details=lusid.models.CreatePortfolioDetails(
      corporate_action_source_id=lusid.ResourceId(
          scope=scope,
          code=ca_source_code,
      ),  
    ),
)

print(
    f"Corporate actions source of {ca_source_code} assigned to portfolio {portfolio_code}"
)

## What transactions make up our holdings?

In [ ]:
# Load a mapping file for DataFrame headers for the build transaction response
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)


build_transactions_response = txn_portfolio_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date="2020-01-01", end_date="2020-12-31"
    ),
    property_keys=["Instrument/default/Name"],
)

build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)
build_transactions_df